# Advent of Code 2022!

# Day 1

In [18]:
# Day 1: Calorie Counting

import numpy as np
import pandas as pd

# read in the file
f = open('data/1_1.csv')
fl = f.readlines()
f.close()

fl = [f.strip('\n') for f in lines]

# Make a running total of calories. 
# When we finish an elf, append their running total to our list
# Elves are separated by blank lines, ('' == end of elf)
n_elf = 0
elf_amounts = []
elf_tot = 0
for f in lines:
    if f == '':
        n_elf += 1
        elf_amounts.append(elf_tot)
        elf_tot = 0
    else:
        elf_tot += int(f)
# make sure the last calorie count is added
elf_amounts.append(elf_tot)

In [19]:
np.max(elf_amounts)

68787

In [20]:
# Now find the sum of the top three

# throw the values into a pandas DataFrame
elfs = pd.DataFrame({'calories': elf_amounts})
# sort by value, take the last three, and sum them up
elfs.sort_values('calories').iloc[-3:,].sum()

calories    198041
dtype: int64

# Day 2

In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/2_1.dat', sep=' ', header=None)
df

,0,1
0,C,Z
1,C,Y
2,B,X
3,A,Z
4,C,Z
...,...,...
2495,B,Z
2496,C,Y
2497,A,Z
2498,A,Z


In [5]:
df.iloc[0,1]

'Z'

In [23]:
def score(row):
    if row[1] == 'X':
        val = 1
        if row[0] == 'C':
            win = 6
        elif row[0] == 'A':
            win = 3
        else:
            win = 0
    elif row[1] == 'Y':
        val = 2
        if row[0] == 'A':
            win = 6
        elif row[0] == 'B':
            win = 3
        else:
            win = 0
    if row[1] == 'Z':
        val = 3
        if row[0] == 'B':
            win = 6
        elif row[0] == 'C':
            win = 3
        else:
            win = 0
    return win + val

# given A (rock), B (paper), or C (scissors)
# return the number corresponding to the loss
def score_loss(target):
    if target == 'A':
        return 3
    elif target == 'B':
        return 1
    else:
        return 2

# given A (rock), B (paper), or C (scissors)
# return the number corresponding to the win
def score_win(target):
    if target == 'A':
        return 2
    elif target == 'B':
        return 3
    else:
        return 1

def score2(row):
    if row[1] == 'X':
        win = 0
        val = score_loss(row[0])
    elif row[1] == 'Y':
        win = 3
        if row[0] == 'A':
            val = 1
        elif row[0] == 'B':
            val = 2
        else:
            val = 3
    else:
        win = 6
        val = score_win(row[0])
    return win + val


In [26]:
scores = []
for row_id in range(df.shape[0]):
    scores.append(score(df.iloc[row_id,]))
np.sum(scores)

9241

In [24]:
scores2 = []
for row_id in range(df.shape[0]):
    scores2.append(score2(df.iloc[row_id,]))
np.sum(scores2)

14610

# Day 3

In [63]:
import string

score_dict = dict()
for i, letter in enumerate(string.ascii_letters):
    score_dict[letter] = i+1

# read in data
with open('data/3_1.dat', 'r') as f:
    lines = f.readlines()

# remove newline character from end of each line
lines = [s.strip('\n') for s in lines]

def split(x):
    # split string in half
    n = len(x)
    return x[slice(0, int(n/2))], x[slice(int(n/2), n)]

def get_match(l1, l2):
    # use list intersection to find the item in both
    return list(set(l1) & set(l2))[0]

def score(x):
    return score_dict[x]


In [64]:
scores = 0
for line in lines:
    l1, l2 = split(line)
    scores += score(get_match(l1, l2))
scores

7831

In [67]:
# part 2, find the sum of values for items common across each group of 3 lines
def match3(l1, l2, l3):
    return list(set(l1) & set(l2) & set(l3))[0]

scores = 0
ii = 0
# iterate through the list (3 at a time) and calculate the sum
while ii < len(lines)-2:
    l1 = lines[ii]
    l2 = lines[ii+1]
    l3 = lines[ii+2]
    scores += score(match3(l1, l2, l3))
    ii += 3 # don't forget to iterate by 3
scores

2683

# Day 4

In [97]:
# Day 4: Camp Cleanup

# There are two columns, which represent two ranges of data
# We want to know if one range every completely encompasses
# the other range
#   e.g. 6-6 is consumed by 4-6

# Let's use pandas and it's built in str functionality
import pandas as pd

df = pd.read_csv('data/4_1.dat', header=None)
df

,0,1
0,8-13,10-65
1,13-23,14-24
2,72-83,73-76
3,37-37,39-73
4,64-94,65-65
...,...,...
995,49-51,50-55
996,7-20,20-60
997,19-99,20-98
998,83-97,45-82


In [101]:
# pull out the start and end values, and convert to int
d = pd.DataFrame()
d[['x1', 'x2']] = df[0].str.split('-', expand=True)
d[['y1', 'y2']] = df[1].str.split('-', expand=True)
d = d.astype(int)
d

,x1,x2,y1,y2
0,8,13,10,65
1,13,23,14,24
2,72,83,73,76
3,37,37,39,73
4,64,94,65,65
...,...,...,...,...
995,49,51,50,55
996,7,20,20,60
997,19,99,20,98
998,83,97,45,82


In [118]:
# function to check fill out the ranges and find overlap
# later updated to specify minimum overlap required
def check_overlap(row, min=None):
    """Check if ranges (row[0]-row[1]) and (row[2]-row[3])
    overlap by at least `min` values
    
    If min is None, match full length of shortest range
    """
    a = np.arange(row[0], row[1]+1)
    b = np.arange(row[2], row[3]+1)

    if min is None:
        if a.shape[0] > b.shape[0]:
            min = b.shape[0]
        else:
            min = a.shape[0]

    m = list(set(a) & set(b))
    if len(m) >= min:
        return 1
    else:
        return 0

In [119]:
# iterate over rows to find complete subsets
matches = 0
for rid in range(d.shape[0]):
    matches += check_overlap(d.iloc[rid,])
matches

526

In [121]:
# iterate over rows to find any overlap
matches = 0
for rid in range(d.shape[0]):
    matches += check_overlap(d.iloc[rid,], min=1)
matches

886

# Day 5: Supply Stacks

In [296]:
def get_stacks():
    stacks_orig = [
        ['W', 'T', 'H', 'P', 'J', 'C', 'F'],
        ['H', 'B','J', 'Z', 'F', 'V', 'R', 'G'],
        ['R', 'T', 'P', 'H'],
        ['T', 'H', 'P', 'N', 'S', 'Z'],
        ['D', 'C', 'J', 'H', 'Z', 'F', 'V', 'N'],
        ['Z', 'D', 'W', 'F', 'G', 'M', 'P'],
        ['P', 'D', 'J', 'S', 'W', 'Z', 'V', 'M'],
        ['S', 'D', 'N'],
        ['M', 'F', 'S', 'Z', 'D']
    ]
    for stack in stacks_orig:
        stack.reverse()
    return stacks_orig

In [302]:
def move(n, from_, to_, stacks):
    for ii in range(n):
        crate = stacks[from_].pop()
        stacks[to_].append(crate)
    return stacks

def multi_move(n, from_, to_, stacks2, debug=False):
    crates = []
    for ii in range(n):
        crates.append(stacks2[from_].pop())

    while len(crates) > 0:
        stacks2[to_].append(crates.pop())
    return stacks2



In [266]:
with open('data/5_2.dat', 'r') as f:
    instr = f.readlines()
instructions = [c.strip('\n').split(' ') for c in instr]
instructions[:3]

[['move', '2', 'from', '8', 'to', '2'],
 ['move', '3', 'from', '9', 'to', '2'],
 ['move', '1', 'from', '3', 'to', '8']]

In [287]:
stacks = get_stacks()
for inst in instructions:
    _, n, _, from_, _, to_ = inst
    stacks = move(int(n), int(from_)-1, int(to_)-1, stacks)

In [288]:
for stack in stacks:
    print(stack[-1], end='')

SPFMVDTZT

In [305]:
stacks2 = get_stacks()
for inst in instructions:
    #print(inst)
    _, n, _, from_, _, to_ = inst
    stacks2 = multi_move(int(n), int(from_)-1, int(to_)-1, stacks2)

In [306]:
for stack in stacks2:
    print(stack[-1], end='')

ZFSJBPRFP

# Day 6

In [1]:
with open('data/6_1.dat', 'r') as f:
    stream = f.readlines()

In [ ]:
buffer = []

for i, s in enumerate(stream[0]):
    buffer.insert(0, s)
    if len(buffer) > 4:
        buffer.pop()
    if len(set(buffer)) == 4:
        break
i+1

In [ ]:
buffer = []

for i, s in enumerate(stream[0]):
    buffer.insert(0, s)
    if len(buffer) > 14:
        buffer.pop()
    if len(set(buffer)) == 14:
        break
i+1

# Day 7

Navigating a terminal to calculate file sizes

In [221]:
class File():
    """Class for files - name (str) and size (int) """

    def __init__(self, name, size):
        self.name = name
        self.size = size

class Folder():
    """"Class for Folders, which have sizes, files, parents, and subfolders"""

    def __init__(self, name, parent=None):
        self.name = name
        self.size = 0
        self.files = {}
        self.subfolders = {}
        self.parent = parent

    def add_folder(self, folder):
        if not folder.name in self.subfolders:
            if folder.parent is None:
                folder.parent = self
            self.subfolders[folder.name] = folder

    def add_file(self, file):
        if not file.name in self.files:
            self.files[file.name] = int(file.size)

    def get_size(self):
        size = 0
        for k, v in self.files.items():
            size += v
        for k, v in self.subfolders.items():
            size += v.get_size()
        return size

    def get_file_sizes(self):
        size = 0
        for k, v in self.files.items():
            size += v
        return size



In [200]:
# load in problem and test files
with open('data/7_1.dat', 'r') as f:
    fl = f.readlines()
fl = [f.strip('\n') for f in fl]

with open('data/7_test.dat', 'r') as ft:
    flt = ft.readlines()
flt = [f.strip('\n') for f in flt]


In [201]:
def run_code(lines):
    """Function to process a text file of terminal commands/outputs"""
    root = Folder('/')
    cur_dir = root
    ii = 0
    while ii < len(lines):
        # check for command
        line = lines[ii]
        if line.startswith('$'):
            if line[2:4] == 'cd':
                _, cmd, arg = line.split(' ')
                if arg == '/':
                    cur_dir = root
                elif arg == '..':
                    cur_dir = cur_dir.parent
                else:
                    if not arg in cur_dir.subfolders:
                        cur_dir.add_folder(Folder(arg, cur_dir))
                        cur_dir = cur_dir.subfolder[arg]
                    else:
                        cur_dir = cur_dir.subfolders[arg]
                    #new_folder = Folder(arg, cur_dir)
                    #cur_dir.add_folder(new_folder)
                    #cur_dir = new_folder
            if line[2:4] == 'ls':
                contents = []
                jj = ii
                while jj < len(lines)-1:
                    if lines[jj+1].startswith('$'):
                        break
                    else:
                        jj += 1
                        first, second = lines[jj].split(' ')
                        if lines[jj].startswith('dir'):
                            cur_dir.add_folder(Folder(second, cur_dir))
                        else:
                            cur_dir.add_file(File(second, first))
                ii = jj
        ii += 1
    return root

def get_sizes(folder, sizes={}, prefix=''):
    folder.size = folder.get_size()
    sizes = {
        f'{prefix}/{folder.name}': folder.get_size()
    }
    if len(folder.subfolders) > 0:
        for k, v in folder.subfolders.items():
            sizes.update(get_sizes(v, sizes, prefix=f'{prefix}/{folder.name}'))
    return sizes

In [202]:
root_test = run_code(flt)
root_test.get_size()

48381165

In [204]:
ftest = get_sizes(root_test)
sums = 0
for k, v in ftest.items():
    if v <= 100000:
        sums += v

print(sums)

95437


In [205]:
root = run_code(fl)
root.get_size()

43956976

In [219]:
sizes = get_sizes(root)

In [220]:
sums = 0
for k, v in sizes.items():
    if v <= 100000:
        sums += v
print(sums)

1908462


In [216]:
# Part 2: find smallest file that can be delete to leave enough space
space_needed = root.size - (70000000 - 30000000)
space_needed

3956976

In [217]:
enough = []
for k, v in fact.items():
    if v >= space_needed:
        enough.append(v)
enough.sort()
print(enough[0])

3979145 43956976


# Day 8

In [357]:
import numpy as np

dat = np.genfromtxt('data/8_1.dat', delimiter=1, dtype=int)

In [358]:
def find_visible(trees, reverse=False):
    """Find all visible trees in given row
    
    reverse (optional) whether to score from back-to-front"""
    smallest = -1
    vis_id = []
    for i, x in enumerate(trees):
        if x > smallest:
            vis_id.append(i)
            smallest = x
    if reverse:
        n = trees.shape[0] - 1
        vis_id = [n - v for v in vis_id]
    return vis_id

def check_forest(forest):
    """For each row and column, check visibility in both directions
    Return a list of tuples (row, col) for each visible tree"""
    visible = []
    n_row, n_col = forest.shape
    for row in range(n_row):
        vis1 = find_visible(forest[row,:])
        vis2 = find_visible(np.flip(forest[row,:]), reverse=True)
        for v in vis1:
            visible.append( (row, v) )
        for v in vis2:
            visible.append( (row, v) )
    for col in range(n_col):
        vis1 = find_visible(forest[:,col])
        vis2 = find_visible(np.flip(forest[:,col]), reverse=True)
        for v in vis1:
            visible.append( (v, col) )
        for v in vis2:
            visible.append( (v, col) )
    return list(set(visible))

In [359]:
# Solution 1: how many visible trees?
res = check_forest(dat)
len(res)

1843

In [360]:
# Part two - find "scene score" for each tree
# Use tall trees 

def gaze(start, dist):
    """Count down dist until value is >= start"""
    count = 0
    for d in dist:
        count += 1
        if d >= start:
            break
    return count

def score_tree(forest, tree):
    """For given tree (row, col), look in each direction, then prodsum counts"""
    row, col = tree
    scores = []
    # look up
    scores.append(gaze(forest[row, col], np.flip(forest[:(row), col])))
    # look left
    scores.append(gaze(forest[row, col], np.flip(forest[row, :(col)])))
    # look down
    scores.append(gaze(forest[row, col], forest[(row+1):, col]))
    # look right
    scores.append(gaze(forest[row, col], forest[row, (col+1):]))
    return scores[0] * scores[1] * scores[2] * scores[3]



In [361]:
scored = []
for tree in res:
    scored.append(score_tree(dat, tree))
np.max(scored)

180000

# Day 9

In [1]:
import numpy as np

with open('data/9_1.dat', 'r') as f:
    fl = f.readlines()
instructions = []
for line in fl:
    direct, dist = line.strip('\n').split(' ')
    instructions.append((direct, int(dist)))

In [2]:
# track head and tail position
class Rope():
    """Class to track head and tail positions as they move"""

    def __init__(self, head_start, tail_start):
        self.head = head_start
        self.tail = tail_start
        self.tail_list = [self.tail.copy()]

    def __str__(self):
        return f'Rope with head at {self.head} and tail at {self.tail}'

    def __repr__(self):
        return f'Rope with head at {self.head} and tail at {self.tail} ({len(self.tail_list)} positions)'

    def check_dist(self):
        """return distance from head to tail"""
        return np.sqrt( (self.head[0] - self.tail[0]) ** 2 + (self.head[1] - self.tail[1]) **2 ) 

    def check_tail(self):
        """If tail is not within 1 of the head, move it"""

        # if head is more more than 1.5 away from tail, tail needs to move
        dist = self.check_dist()
        if dist > 1.5:
            xdif = self.head[0] - self.tail[0]
            ydif = self.head[1] - self.tail[1]
            if dist == 2: # non-diagonal
                if xdif == 0: # column move
                    self.tail[1] += (ydif/2)
                else:
                    self.tail[0] += (xdif/2)
            else: # diagonal move
                self.tail[1] += (ydif/np.abs(ydif))
                self.tail[0] += (xdif/np.abs(xdif))
            
            if not self.tail in self.tail_list:
                self.tail_list.append(self.tail.copy())

    def step(self, direction):
        #print(f'Taking step {direction}')
        if direction == 'R':
            self.head[0] += 1
        elif direction == 'L':
            self.head[0] -= 1
        elif direction == 'U':
            self.head[1] += 1
        elif direction == 'D':
            self.head[1] -= 1
        else:
            raise ValueError(f'Unrecognized direction {direction} (must be R, L, D or U)')
        self.check_tail()
        #print(f'Head is {self.head} and Tail is {self.tail}')

    def move(self, inst):
        d, nstep = inst
        for i in range(nstep):
            self.step(d)

In [3]:
rope = Rope([0, 0], [0, 0])
rope

Rope with head at [0, 0] and tail at [0, 0] (1 positions)

In [4]:
for inst in instructions:
    rope.move(inst)
rope

Rope with head at [-173, 503] and tail at [-173.0, 502.0] (6391 positions)

In [5]:
# track head and tail position
# track head and tail position
class Rope2():
    """Class to track head and tail positions as they move"""

    def __init__(self, head_start, tail_start):
        self.head = head_start
        self.tail = tail_start
        self.tail_list = [tail_start.copy()]

    def __str__(self):
        return f'Rope with head at {self.head} and tail at {self.tail}'

    def __repr__(self):
        return f'Rope with head at {self.head} and tail at {self.tail} ({len(self.tail_list)} positions)'

    def check_dist(self):
        """return distance from head to tail"""
        return np.sqrt( (self.head[0] - self.tail[0]) ** 2 + (self.head[1] - self.tail[1]) **2 ) 

    def check_tail(self):
        """If tail is not within 1 of the head, move it"""

        # if head is more more than 1.5 away from tail, tail needs to move
        dist = self.check_dist()
        if dist > 1.5: # not touching
            xdif = self.head[0] - self.tail[0]
            if xdif == 0:
                xstep = 0
            else:
                xstep = xdif / np.abs(xdif)
            ydif = self.head[1] - self.tail[1]
            if ydif == 0:
                ystep = 0
            else:
                ystep = ydif / np.abs(ydif)
            # update tail positions
            self.tail[0] += xstep
            self.tail[1] += ystep
            
            if not self.tail in self.tail_list:
                self.tail_list.append(self.tail.copy())

    def step(self, direction):
        #print(f'Taking step {direction}')
        if direction == 'R':
            self.head[0] += 1
        elif direction == 'L':
            self.head[0] -= 1
        elif direction == 'U':
            self.head[1] += 1
        elif direction == 'D':
            self.head[1] -= 1
        else:
            raise ValueError(f'Unrecognized direction {direction} (must be R, L, D or U)')
        self.check_tail()
        #print(f'Head is {self.head} and Tail is {self.tail}')

    def move(self, inst):
        d, nstep = inst
        for i in range(nstep):
            self.step(d)

In [6]:
import matplotlib.pyplot as plt
def plot_rope(knots):
    heads = [k.head for k in knots]
    tails = [k.tail for k in knots]
    max_x = np.max([np.max(h) for h in heads])
    max_y = np.max([np.max(h) for h in tails])
    min_x = np.min([np.min(h) for h in heads])
    min_y = np.min([np.min(h) for h in tails])
    
    fig, ax = plt.subplots(figsize=(5,5))
    ax.set_xlim(-15, 15)
    ax.set_ylim(-15, 15)
    positions = []
    for i, k in enumerate(knots):
        if not k.head in positions:
            if i == 0:
                lab = 'H'
            else:
                lab = f'{i}'
            ax.annotate(lab, k.head)
            positions.append(k.head)
    if not [0, 0] in positions:
        ax.annotate('s', (0, 0))


    for ii in range(-20, 20):
        if ii % 5 == 0:
            ax.axvline(ii, alpha=.35)
            ax.axhline(ii, alpha=.35)
        else:
            ax.axvline(ii, alpha=.1)
            ax.axhline(ii, alpha=.1)

    ax.set_box_aspect(1)

In [10]:
# Part 2: rope of multiple knots
def get_knots():
    new_knots = [Rope2([0, 0], [0, 0]) for i in range(9)]
    for i, k in enumerate(new_knots[1:]):
        k.head = new_knots[i].tail
    return new_knots

def multi_move(knots, inst):
    d, nstep = inst
    for ii in range(nstep):
        knots[0].step(d)
        for k in knots[1:]:
            k.check_tail()
    return knots


In [11]:
knots = get_knots()

In [12]:
knots = get_knots()
for inst in instructions:
    knots = multi_move(knots, inst)
knots[-1]

Rope with head at [-173.0, 495.0] and tail at [-174.0, 495.0] (2593 positions)